<a href="https://colab.research.google.com/github/shkim0116/Lecture-Recommendation-NLP/blob/main/preprocessing_tokenize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install contextualized-topic-models==2.2.0
!pip install pyldavis
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt, Mecab
from tqdm import tqdm

In [ ]:
label = pd.read_csv("/content/drive/Shareddrives/NLP모델링/labeling - labeling (1).csv", encoding = 'cp949')

#null값 0.5로 채우기
label.fillna(0.5, inplace = True)

#칼럼 이름 간단하게
label.columns = ['암것','강의명/교수명','강의평','성적','로드','강의력','성향','족보','자료']

#target data를 하나의 vector로 만들기
target = [[label.iloc[i]['성적'],label.iloc[i]['로드'], label.iloc[i]['강의력'], label.iloc[i]['성향'], label.iloc[i]['족보'], label.iloc[i]['자료']]\
          for i in range(len(label))]

#target 칼럼 추가
label['target'] = target

#새로운 데이터프레임
label_target = label[['강의명/교수명','target']]

In [ ]:
import os

path = '/content/drive/Shareddrives/NLP모델링/강의평 데이터/크롤링 통합'
file_list = os.listdir(path)

#강의평 불러오기
df = pd.DataFrame()
for i in file_list:
    data = pd.read_csv(path+ '/' + i)
    df = pd.concat([df,data])

#index reset
review = df.reset_index(drop = True)
review_dropd = review.drop_duplicates(keep = 'first')

#'강의명/교수명' 칼럼 생성
review_copy = review_dropd.copy()
review_copy['강의명/교수명'] = review_copy['강의명']+'/'+review_copy['교수명']

#강의평 개수가 20개 이상인 것들만 강의평 불러오기
save = pd.merge(review_copy[['강의명/교수명']].drop_duplicates(keep = 'first'),review_copy.groupby('강의명/교수명').count()[['강의평']], on = '강의명/교수명')
save1 = save[save['강의평'] >= 20]
over20review = pd.merge(review_copy,save1, how = 'inner',on = '강의명/교수명')
over20review_1 = over20review[['강의명/교수명','강의평_x']]
over20review_1.columns = ['강의명/교수명','강의평']

In [ ]:
#강의평과 target vector merge
dataset = pd.merge(over20review_1, label_target, how = 'inner', on='강의명/교수명')
dataset.head()

,강의명/교수명,강의평,target
0,대학기본영어Ⅱ/오진숙,어렵지 않은 내용으로 탄탄한 기본기를 다져주는 유익한 수업,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]"
1,대학기본영어Ⅱ/오진숙,교수님 너무 귀여우심... 사랑해요\n자잘한 과제들 귀찮긴 했지만 시간투자 그렇게 ...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]"
2,대학기본영어Ⅱ/오진숙,완전 좋음 2학기 때 내가 살아있던 이유.. 기말 때 갑자기 시험을 세번 본 거는 ...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]"
3,대학기본영어Ⅱ/오진숙,영어 말하는 데에 자신감을 얻고 싶으면 추천 다만 과제는 자잘한게 많고 필자는 1학...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]"
4,대학기본영어Ⅱ/오진숙,수강당시에는 조모임 비율도 높았고 발음 중시하시는건 왜 그러시는지 잘 이해가지 않지...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]"


In [ ]:
#강의평 중 한글만 불러오기
dataset['강의평'] = dataset['강의평'].str.replace("[^가-힣ㄱ-하-ㅣ ]","")
dataset.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


,강의명/교수명,강의평,target
0,대학기본영어Ⅱ/오진숙,어렵지 않은 내용으로 탄탄한 기본기를 다져주는 유익한 수업,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]"
1,대학기본영어Ⅱ/오진숙,교수님 너무 귀여우심 사랑해요자잘한 과제들 귀찮긴 했지만 시간투자 그렇게 오래 걸리...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]"
2,대학기본영어Ⅱ/오진숙,완전 좋음 학기 때 내가 살아있던 이유 기말 때 갑자기 시험을 세번 본 거는 당황스...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]"
3,대학기본영어Ⅱ/오진숙,영어 말하는 데에 자신감을 얻고 싶으면 추천 다만 과제는 자잘한게 많고 필자는 학기...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]"
4,대학기본영어Ⅱ/오진숙,수강당시에는 조모임 비율도 높았고 발음 중시하시는건 왜 그러시는지 잘 이해가지 않지...,"[1.0, 0.5, 0.5, 1.0, 0.0, 0.5]"


In [ ]:
okt = Okt()
stop = ['Josa','Suffix','Punctuation', 'Foreign', 'Number']

tokenized = []
for review in tqdm(dataset['강의평']):
    t = okt.pos(review, stem = True)
    t = [w[0] for w in t if w[1] not in stop]
    tokenized.append(t)

100%|██████████| 27142/27142 [05:08<00:00, 87.97it/s]


In [ ]:
dataset_reviewtoken = dataset.copy()
dataset_reviewtoken['강의평토큰화'] = tokenized

In [ ]:
tokenized_reduce = [i for i in tokenized if len(i) > 1]

In [ ]:
dataset_reduced = dataset_reviewtoken[dataset_reviewtoken['강의평토큰화'].isin(tokenized_reduce)]

In [ ]:
# dataset_reduced.to_csv('/content/drive/Shareddrives/NLP모델링/코드 연습/문서/reviewtokenized.csv')